## Kemal Demirel: 191104091

## Kadir Çağan Eren: 

In [55]:
import glob
import pathlib
import cv2
import numpy as np 
import pandas as pd 
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


## Read all dataset with images

In [10]:
def traverse_directory(path):
    entered = False
    dataset = []
    labels = []
    for dirpath, dirnames, filenames in os.walk(path):
        if not entered:
            entered = True    
        for filename in filenames:
            file_path = os.path.join(dirpath, filename)
            if(file_path.endswith(".jpg")):
                img = cv2.imread(file_path)
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (64, 64))
                dataset.append(img)
                tmp_path = dirpath
                label = tmp_path[tmp_path.find("train/")+len("train/"):]
                labels.append(label)
    return np.array(dataset), np.array(labels)

In [11]:
train_images, train_labels = traverse_directory(os.getcwd()+"/dataset/train")

In [41]:
def extract_features_sift(image):
    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    # Initialize a SIFT object and detect keypoints and descriptors
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(gray, None)

    # Return the descriptors as a 1D vector
    return descriptors.flatten()


In [43]:
def extract_features_histogram(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    # Compute a 3D color histogram using 8 bins per channel
    hist = cv2.calcHist([hsv], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])

    # Normalize the histogram to have unit L1 norm
    hist = cv2.normalize(hist, hist)

    # Flatten the histogram into a 1D vector
    return hist.flatten()


In [44]:
extract_features_histogram(train_images[0])

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 7.01274956e-04, 9.93355989e-01,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 3.85701237e-03, 9.11657419e-03,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       7.01274956e-04, 1.05191243e-03, 3.50637478e-04, 1.22723114e-02,
       0.00000000e+00, 0.00000000e+00, 3.50637478e-04, 3.50637478e-04,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.71812363e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 3.50637478e-04,
       7.01274956e-04, 1.05191243e-03, 7.01274956e-04, 3.36611979e-02,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.05191243e-03,
       1.40254991e-03, 0.00000000e+00, 3.15573718e-03, 6.80236742e-02,
       0.00000000e+00, 0.00000000e+00, 3.50637478e-04, 1.40254991e-03,
       1.40254991e-03, 3.50637478e-04, 4.20764973e-03, 4.55828710e-03,
      

In [48]:
#test_images, test_labels = traverse_directory(os.getcwd()+"/dataset/test")
X_train, X_test, y_train, y_test = train_test_split(train_images, train_labels, test_size=0.2)

In [49]:
X_train_features = np.array([extract_features_histogram(image) for image in X_train])
X_test_features = np.array([extract_features_histogram(image) for image in X_test])

In [56]:
# Train a classifier
def train_classifier(X_train, y_train):
    svm = SVC(kernel='linear')
    svm.fit(X_train, y_train)
    return svm

In [57]:
# Evaluate the classifier
def evaluate_classifier(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    return accuracy, precision, recall, f1

In [58]:

# Predict new images
def predict(classifier, image):
    feature = extract_features_histogram(image)
    prediction = classifier.predict([feature])
    return prediction[0]

In [59]:
classifier = train_classifier(X_train_features, y_train)

In [66]:
accuracy, precision, recall, f1 = evaluate_classifier(classifier, X_test_features, y_test)

# Predict new images
new_image = cv2.imread("/Users/kemaldemirel/Desktop/Lectures/Bil 468/Project/Computer-Vision-Fruit-Vegetables-Detection/dataset/test/grapes/Image_1.jpg")
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image = cv2.resize(new_image, (64, 64))
prediction = predict(classifier, new_image)
print(f1)

0.19830597047350615


/Users/kemaldemirel/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
